# Preparation of Flux-Free Kitaev Honeycomb lattice state
## In TORIC CODE SPACE

2 honeycombs

N = 14 spins

Hamiltonian: $H = -J_x \sum_{\langle i,j \rangle_x} \sigma_i^x \sigma_j^x
    -J_y \sum_{\langle i,j \rangle_y} \sigma_i^y \sigma_j^y
    -J_z \sum_{\langle i,j \rangle_z} \sigma_i^z \sigma_j^z$ 

Step 1: prepare product state $|00..>$ of n = N/2 spins

In [19]:
import numpy as np
import scipy
from scipy import sparse
import importlib

We are doing this mapping: 

$\ket{\uparrow \uparrow} = \ket{0} , \quad\ket{\downarrow \downarrow} = \ket{1} $


![Graphene]()



In [2]:
n_spins = 7
e1 = sparse.csr_array([[1],[0]])
psi = e1
for _ in range(n_spins-1):
    psi = sparse.kron(psi,e1, format='csr')

print(psi.shape)


assert psi.shape[0]== 2**7

(128, 1)


In [17]:
print(psi)

  (0, 0)	1


In [3]:
print(2**7) 

128


Identify a representative qubit in each plaquette: we identify qubit 0 and 6

We create operator that applies hadamard only on these two representative qubits

In [4]:
def Hadamard(qubit1, qubit2, n):
    assert qubit1 < qubit2
    Id = sparse.csr_array(np.eye(2))
    H_small = sparse.csr_array([[1./np.sqrt(2),1./np.sqrt(2)], [1./np.sqrt(2),-1./np.sqrt(2)]])
    op_list = [Id]*n
    op_list[qubit1] = H_small
    op_list[qubit2] = H_small
    
    H = op_list[0]

    for op in op_list[1:]:
        H = sparse.kron(H,op, format='csr')
    return H
    

TEST: given qubit1 = 0, qubit2 = 1, and n = 3: $ \quad H \ket{000} = \ket{++0}$, which should be a sparse matrix with $1/\sqrt(2)$ at position (0,4,2,6)   

In [5]:
psitest = e1
for _ in range(2):
    psitest = sparse.kron(psitest,e1, format='csr')

Htest = Hadamard(0,1,3)
psitestnew = Htest*psitest

print(psitestnew)

  (0, 6)	0.4999999999999999
  (0, 4)	0.4999999999999999
  (0, 2)	0.4999999999999999
  (0, 0)	0.4999999999999999


In [ ]:
qubit1 = 0
qubit2 = 4
H = Hadamard(qubit1,qubit2,n_spins)
print(H.shape)
psi1 = H @ psi.copy()

print(psi1.shape)
print(psi1)

(128, 128)
(128, 1)
  (0, 0)	0.4999999999999999
  (4, 0)	0.4999999999999999
  (64, 0)	0.4999999999999999
  (68, 0)	0.4999999999999999


In [21]:
import function as func
importlib.reload(func)

<module 'function' from '/home/t30/pol/go56vod/Desktop/MasterThesis/function.py'>

In [22]:
psi1_new = func.mapping(psi1, n_spins)

[ 0  4 64 68]
[0.5 0.5 0.5 0.5]
Original Sparse Array:
  (0, 0)	0.4999999999999999
  (4, 0)	0.4999999999999999
  (64, 0)	0.4999999999999999
  (68, 0)	0.4999999999999999

New Sparse Array:
  (0, 0)	0.4999999999999999
  (48, 0)	0.4999999999999999
  (12288, 0)	0.4999999999999999
  (12336, 0)	0.4999999999999999


Create op. that applies CNOT on each plaquette with representative qubits as control qubits and all other qubits of plaquette as target ones

In [23]:
def CNOT(qubit1, qubit2, n, l): #l = spins per plaquette, n = tot spins
    assert qubit1 < 3
    assert 3 < qubit2
    # Id = sparse.csr_array(np.eye(2))
    # X = sparse.csr_array([[0.,1.],[1.,0.]])
    # Proj_0 = sparse.csr_array([[1.,0.],[0.,0.]]) #|0><0|
    # Proj_1 = sparse.csr_array([[0.,0.],[0.,1.]]) #|1><1|
    
    newId = sparse.csr_array(np.eye(2**(n-l)))
    CNOT_p1 = sparse.kron(plaquetteCNOT(qubit1,l),newId,'csr')
    CNOT_p2 = sparse.kron(newId,plaquetteCNOT(qubit2-l+1,l), 'csr') #we map qubit 3 -> 0, 4->1, 5->2, 6->3
    # even if geometry is not conserved, it still works because CNOT applied on this 
    #sequence of hilbert spaces in this order does not depend on geometry!
    # print(CNOT_p2 @ CNOT_p1)
    # print(CNOT_p1 @ CNOT_p2)
    assert np.array_equal((CNOT_p2 @ CNOT_p1).toarray(), (CNOT_p1 @ CNOT_p2).toarray()), "Sparse arrays are not equal"
    return CNOT_p2 @ CNOT_p1

def plaquetteCNOT(qubit, l):
    Id = sparse.csr_array(np.eye(2))
    X = sparse.csr_array([[0.,1.],[1.,0.]])
    Proj_0 = sparse.csr_array([[1.,0.],[0.,0.]]) #|0><0|
    Proj_1 = sparse.csr_array([[0.,0.],[0.,1.]]) #|1><1|

    op_list_id = [Id]*l
    op_list_id[qubit] = Proj_0

    op_list_x = [X]*l
    op_list_x[qubit] = Proj_1

    CNOT1 = op_list_id[0]
    CNOT2 = op_list_x[0]

    for op1, op2 in zip(op_list_id[1:],op_list_x[1:]):
        CNOT1 = sparse.kron(CNOT1, op1, 'csr')
        CNOT2 = sparse.kron(CNOT2, op2, 'csr')
    return CNOT1 + CNOT2

In [24]:
#TEST FOR plaquetteCNOT: it works
CNOT_test = plaquetteCNOT(0,2)
print(CNOT_test*([1,0,1,0]))

[1. 0. 0. 1.]


In [25]:
CNOT_op = CNOT(qubit1, qubit2, n_spins, 4)
#print(CNOT_op.toarray())
psi2 =  CNOT_op @ psi1.copy()
#print(psi1.shape)
#print(CNOT_op.shape)
#print(psi2.shape)
print(psi2.toarray().flatten())

[0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.  0.  0.  0.  0.
 0.  0. ]


Now we want to convert this state from toric code space to Honeycomb space! Mapping written on ipad

In [26]:
psi2_new = func.mapping(psi2, n_spins)

[  0  15 119 120]
[0.5 0.5 0.5 0.5]
Original Sparse Array:
  (0, 0)	0.4999999999999999
  (15, 0)	0.4999999999999999
  (119, 0)	0.4999999999999999
  (120, 0)	0.4999999999999999

New Sparse Array:
  (0, 0)	0.4999999999999999
  (255, 0)	0.4999999999999999
  (16191, 0)	0.4999999999999999
  (16320, 0)	0.4999999999999999


Next step: apply $U_a$ to horizontal sites: qubit = 1,3,6, $U_b$ to vertical ones: qubit = 0,2,4,5 $U = \prod_{hor} U_a \prod_{vert} U_b$

In [27]:
def U_a_full(n):
    U_a = sparse.csr_array([[(1.+1.j)/2.,(1.+1.j)/2.],[(-1.+1.j)/2.,(1.-1.j)/2.]])
    Id = sparse.csr_array(np.eye(2))
    op_list = [Id]*n
    op_list[1] = U_a
    op_list[3] = U_a
    op_list[6] = U_a

    U = op_list[0]

    for op in op_list[1:]:
        U = sparse.kron(U,op, format='csr')
    return U

def U_b_full(n):
    U_b = sparse.csr_array([[1./np.sqrt(2)*(1.-1.j),0.],[0.,1./np.sqrt(2)*(1.+1.j)]])
    Id = sparse.csr_array(np.eye(2))
    op_list = [U_b]*n
    op_list[1] = Id
    op_list[3] = Id
    op_list[6] = Id

    U = op_list[0]

    for op in op_list[1:]:
        U = sparse.kron(U,op, format='csr')
    return U


In [28]:
psi_final = U_a_full(n_spins) @ U_b_full(n_spins) @ psi2.copy()
print(psi_final.shape)
print(psi_final.toarray().flatten())

(128, 1)
[ 0.125-0.125j  0.125+0.125j  0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j    -0.125+0.125j  0.125+0.125j  0.125+0.125j -0.125+0.125j
  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.125+0.125j
  0.125-0.125j  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     0.125+0.125j -0.125+0.125j  0.   +0.j
  0.   +0.j     0.   +0.j     0.   +0.j    -0.125-0.125j -0.125+0.125j
 -0.125+0.125j -0.125-0.125j  0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j    -0.125+0.125j  0.125+0.125j  0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
  0.   +0.j     0.   +0.j     

We now have to verify if this state has plaquette terms wp = +1

In [ ]:
def Op_full(O,pos, n):
    Id = sparse.csr_array(np.eye(2))
    op_list = [Id]*n
    op_list[pos] = O

    fullOp = op_list[0]

    for op in op_list[1:]:
        fullOp = sparse.kron(Op,op, format='csr')
    return fullOp   

In [30]:
X = sparse.csr_array([[0.,1.],[1.,0.]])
Y = sparse.csr_array([[0.,-1.j],[1.j,0.]])
Z = sparse.csr_array([[1.,0.],[0.,-1.]])
I = sparse.csr_array(np.eye(2))

1st: plaquette term written in toric code space

In [32]:
X_u = Op_full(X, 0, n_spins)
Y_l = Op_full(Y, 1, n_spins)
X_d = Op_full(X, 2, n_spins)
Y_r = Op_full(Y, 3, n_spins)
W_tilde = X_u @ Y_l @ X_d @ Y_r
print(W_tilde.shape)

(128, 128)


In [37]:
# Compute the complex conjugate transpose of psi_final
psi_final_dagger = psi_final.conjugate().transpose()

# Compute the expectation value
expectation_value = (psi_final_dagger @ W_tilde @ psi_final)[0,0]

# Print the result
print("Expectation Value:", expectation_value)

#I get 0 also replacing X with Z!!!

Expectation Value: 0j


In [ ]:
#form x,y,z full operators to build plaquette term and once you have written state in Honeycomb space check if plaquette terms are +1!